In [1]:

# coding: utf-8

# In[1]:


import sys, os
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '../../game')))
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from coach import Coach
from NeuralNetworkWrapper import NNetWrapper
from train import Train
from multiprocessing import Pool
from tensorboardX import SummaryWriter
from GOLAI.arena import Arena
from game import Game
import numpy as np
import random
import torch
import time
from utils import *

def createRandomOpp():
    
    opponents = []
    value = list(range(0, args.vocabLen))
    for i in range(args.numEps):
        opponents.append(np.random.choice(value, args.predictionLen).tolist())
    
    return opponents

        
def createCoachData(nnet, opponents, iteration):
    random.shuffle(opponents)
    opponents = np.array(opponents)
    opponents = np.split(opponents, args.cpus)
    gpuId = np.arange(args.gpus)
    coachData = []
    for i in range(args.cpus):
        coachData.append([nnet, gpuId[0], iteration, opponents[i]])
        gpuId = np.roll(gpuId, 1)
    
    return coachData
    
    
def generateData(variables):    
    nnet = variables[0]
    gpuId = variables[1]
    iteration = variables[2]
    trainingData = variables[3]
    
    g = Game(Arena(args.gameWidth, args.gameHeight))
    c = Coach(g, nnet, opponents)
    
    return c.learn()



if __name__=="__main__":
    #args.program_dir = os.path.join(args.output_dir, args.time)
    nnet = NNetWrapper()
    t = Train(nnet)
    writer = SummaryWriter()
    opponents = createRandomOpp()
    
    if args.load_model:
        nnet.load_checkpoint(args.load_folder_file[0], args.load_folder_file[1])
    
    if args.load_model:
        print("Load trainExamples from file")
        t.loadTrainExamples()
        
    for i in range(args.numIters):
        coachData = createCoachData(nnet, opponents, i)
        #os.makedirs(os.path.join(args.program_dir, str(i)))
        p = Pool(processes = args.cpus)
        data = p.starmap(generateData, coachData)
        p.close()
        
        data = np.array(data)
        wins = sum(data[:, 2].tolist())
       
        writer.add_scalars('zero_10/winning', {'winning': wins}, i)
        nnet = t.run(data[0])
        opponents = data[:, 1]


KeyError: '__getstate__'